# Create a catalogue of tweets in mongodb

In [1]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
#!pip install pymongo

In [3]:
import json
from hdfs import InsecureClient
from pymongo import MongoClient

# MONGODB
mongo_url = 'mongodb://hadoop-vm.internal.cloudapp.net:27017/ca2.tweets'
database_name = 'ca2'
collection_name = 'tweets'

# HADOOP
hdfs_url = 'http://hadoop-vm.internal.cloudapp.net:9870'
hdfs_directory = "/twitter"

## Connect and create database and tweets collection
Include index on field id for performance

# Read tweets from hadoop

In [4]:
# Connect to Hadoop
client = InsecureClient(hdfs_url, user='hduser')

# Connect to MongoDB
mongo_client = MongoClient(mongo_url)
db = mongo_client[database_name]
collection = db[collection_name]

# List JSON files in Hadoop directory
json_files = client.list(hdfs_directory)

# Read JSON files and insert into MongoDB
for file_name in json_files:
    if file_name.endswith('.json'):
        file_path = f"{hdfs_directory}/{file_name}"
        print(f"processing {file_path}")
        with client.read(file_path) as file:
            for line in file:
                try:
                    tweet = json.loads(line)
                    tweet_p = {
                        '_id': tweet['id'],
                        'text': tweet.get('text'),
                        'timestamp_ms': tweet.get('timestamp_ms'),
                        'geo': tweet.get('geo'),
                        'coordinates': tweet.get('coordinates'),
                    }
                    if 'extended_tweet' in tweet:
                        tweet_p['full_text'] = tweet['extended_tweet'].get('full_text')

                    collection.update_one({'_id': tweet['id']}, {'$set': tweet_p}, upsert=True)
                except:
                    continue


{"exception":{"exception_class":"java.lang.IllegalArgumentException","exception_message":"Too large frame: 1586112597084667896","stacktrace":"java.lang.IllegalArgumentException: Too large frame: 1586112597084667896\n\tat org.sparkproject.guava.base.Preconditions.checkArgument(Preconditions.java:119)\n\tat org.apache.spark.network.util.TransportFrameDecoder.decodeNext(TransportFrameDecoder.java:148)\n\tat org.apache.spark.network.util.TransportFrameDecoder.channelRead(TransportFrameDecoder.java:98)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:365)\n\tat io.netty.channel.AbstractChannelHandlerContext.fireChannelRead(AbstractChannelHandlerContext.java:357)\n\tat io.netty.channel.DefaultChannelPipeline$HeadContext.channelRead(DefaultChannelPipeline.java:1410)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeCha

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 43888)
Traceback (most recent call last):
  File "/anaconda/envs/py38_default/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/anaconda/envs/py38_default/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/anaconda/envs/py38_default/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/anaconda/envs/py38_default/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/dsvm/tools/spark/current/python/pyspark/accumulators.py", line 260, in handle
    poll(authenticate_and_accum_updates)
  File "/dsvm/tools/spark/current/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/dsvm/tools/spark/current/python/pyspark/accumula

KeyboardInterrupt: 